In [ ]:
# Example: Batch prediction on a folder
# Replace with your folder path
image_folder = "path/to/your/images_folder"

# Uncomment below and update the path to test
# results = predict_batch(image_folder)
# display_batch_results(results)

## Example Usage: Batch Predictions

Predict on all images in a folder at once.

In [ ]:
# Example: Predict on a single image
# Replace with your image path
image_path = "path/to/your/image.jpg"

# Uncomment below and update the path to test
# result = predict_image(image_path)
# display_prediction(result)

## Example Usage: Single Image Prediction

Replace the image path with your own image and run the cell below!

In [ ]:
def predict_batch(image_folder):
    """
    Make predictions on all images in a folder
    
    Args:
        image_folder: Path to folder containing images
        
    Returns:
        List of prediction results
    """
    image_folder = Path(image_folder)
    if not image_folder.is_dir():
        raise ValueError(f"Not a valid directory: {image_folder}")
    
    # Find all image files
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
    image_files = [f for f in image_folder.iterdir() 
                   if f.suffix.lower() in image_extensions]
    
    if not image_files:
        print(f"No images found in {image_folder}")
        return []
    
    print(f"Found {len(image_files)} images in {image_folder}")
    print("Processing...\n")
    
    results = []
    for image_file in image_files:
        try:
            result = predict_image(str(image_file))
            results.append(result)
            print(f"✓ {image_file.name:30s} → {result['predicted_class'].upper():6s} ({result['confidence']:.2%})")
        except Exception as e:
            print(f"✗ {image_file.name:30s} → ERROR: {str(e)[:40]}")
            results.append({
                'image_path': str(image_file),
                'predicted_class': 'error',
                'confidence': 0.0,
                'error': str(e)
            })
    
    print(f"\nProcessed {len(results)} images")
    return results


def display_batch_results(results):
    """
    Display results from batch prediction
    
    Args:
        results: List of prediction results from predict_batch()
    """
    import pandas as pd
    
    # Create summary dataframe
    data = []
    for r in results:
        if 'error' not in r:
            data.append({
                'Filename': Path(r['image_path']).name,
                'Predicted': r['predicted_class'].upper(),
                'Confidence': f"{r['confidence']:.2%}"
            })
    
    if data:
        df = pd.DataFrame(data)
        print(df.to_string(index=False))
        print(f"\nTotal: {len(data)} successful predictions")
    else:
        print("No successful predictions")

print("✓ Batch prediction functions defined")

## 6. Batch Predictions on Multiple Images

Process multiple images at once and return predictions for each.

In [ ]:
def display_prediction(result):
    """
    Display prediction results with visualization
    
    Args:
        result: Dictionary returned from predict_image()
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Display image
    ax1.imshow(result['original_image'])
    ax1.axis('off')
    ax1.set_title("Input Image", fontsize=14, fontweight='bold')
    
    # Display prediction
    pred_class = result['predicted_class'].upper()
    confidence = result['confidence']
    ax2.text(0.5, 0.7, f"Predicted: {pred_class}", 
             ha='center', fontsize=20, fontweight='bold',
             transform=ax2.transAxes, color='green')
    ax2.text(0.5, 0.6, f"Confidence: {confidence:.2%}", 
             ha='center', fontsize=16,
             transform=ax2.transAxes)
    
    # Display all probabilities as bar chart
    probs = result['all_probabilities']
    classes_list = list(probs.keys())
    probs_list = list(probs.values())
    colors = ['green' if cls == pred_class else 'lightblue' for cls in classes_list]
    
    ax2.bar(range(len(classes_list)), probs_list, color=colors, alpha=0.7)
    ax2.set_xticks(range(len(classes_list)))
    ax2.set_xticklabels([c.upper() for c in classes_list])
    ax2.set_ylabel("Probability", fontsize=12)
    ax2.set_ylim(0, 1)
    ax2.grid(axis='y', alpha=0.3)
    ax2.set_title("Confidence Scores", fontsize=14, fontweight='bold')
    
    # Add probability text on bars
    for i, (cls, prob) in enumerate(probs.items()):
        ax2.text(i, prob + 0.02, f"{prob:.2%}", ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\n{'='*50}")
    print(f"Image: {Path(result['image_path']).name}")
    print(f"Predicted Class: {pred_class}")
    print(f"Confidence: {confidence:.2%}")
    print(f"\nAll Probabilities:")
    for cls, prob in result['all_probabilities'].items():
        print(f"  {cls.upper():6s}: {prob:.4f} ({prob:.2%})")
    print(f"{'='*50}\n")

print("✓ Display function defined")

## 5. Display Predictions with Confidence Scores

Visualize predictions with confidence scores for all classes.

In [ ]:
def predict_image(image_path):
    """
    Predict the animal class in an image
    
    Args:
        image_path: Path to the image file
        
    Returns:
        Dictionary with prediction results
    """
    # Load and preprocess image
    img_array, original_img = load_and_preprocess_image(image_path)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Get results
    pred_probs = predictions[0]
    pred_class_idx = np.argmax(pred_probs)
    pred_class = CLASSES[pred_class_idx]
    pred_confidence = float(pred_probs[pred_class_idx])
    
    return {
        'image_path': image_path,
        'predicted_class': pred_class,
        'confidence': pred_confidence,
        'all_probabilities': {cls: float(prob) for cls, prob in zip(CLASSES, pred_probs)},
        'original_image': original_img
    }

print("✓ Prediction function defined")

## 4. Make Predictions on New Images

Use the loaded model to classify images. The model outputs probability scores for each class.

In [ ]:
# Configuration
IMG_SIZE = (224, 224)
CLASSES = ['bird', 'cat', 'dog']

def load_and_preprocess_image(image_path, show_image=False):
    """
    Load and preprocess image for model prediction
    
    Args:
        image_path: Path to the image file
        show_image: If True, display the loaded image
        
    Returns:
        Preprocessed image array with batch dimension
    """
    # Load image
    img = keras_image.load_img(image_path, target_size=IMG_SIZE)
    
    # Convert to array
    img_array = keras_image.img_to_array(img)
    
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    
    # Apply EfficientNetB0 preprocessing
    img_array = preprocess_input(img_array)
    
    # Display image if requested
    if show_image:
        # For display, we need to denormalize
        display_img = keras_image.load_img(image_path, target_size=IMG_SIZE)
        plt.figure(figsize=(6, 6))
        plt.imshow(display_img)
        plt.axis('off')
        plt.title(f"Image: {Path(image_path).name}")
        plt.tight_layout()
        plt.show()
    
    return img_array, img

print("✓ Image preprocessing function defined")

## 3. Define Image Loading and Preprocessing Function

Create a function to load images, resize, normalize, and prepare them for prediction.

In [ ]:
# Define model directory path
current_dir = Path.cwd()
model_dir = current_dir / 'classifier' / 'model'

# Model file candidates
model_candidates = [
    'CatDogBird_model.h5',
    'CatDogBird_model_Adv.h5',
    'model.h5'
]

# Load the model
model = None
for model_name in model_candidates:
    model_path = model_dir / model_name
    if model_path.exists():
        print(f"Loading model from {model_path}...")
        model = tf.keras.models.load_model(str(model_path))
        print(f"✓ Model loaded successfully!")
        break

if model is None:
    # Try loading weights if full model not found
    weights_path = model_dir / 'ANN_model.weights.h5'
    if weights_path.exists():
        print(f"Full model not found. Loading weights only from {weights_path}...")
        # Need to rebuild architecture first
        from tensorflow.keras import layers, models
        from tensorflow.keras.applications import EfficientNetB0
        
        base_model = EfficientNetB0(include_top=False, weights='imagenet', 
                                    input_shape=(224, 224, 3), pooling='max')
        base_model.trainable = True
        model = models.Sequential([
            base_model,
            layers.Dense(132, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(3, activation='softmax')
        ])
        model.load_weights(str(weights_path))
        print(f"✓ Model architecture and weights loaded successfully!")
    else:
        raise FileNotFoundError(f"No model files found in {model_dir}")

# Display model info
print(f"\nModel Type: {model.__class__.__name__}")
print(f"Model Layers: {len(model.layers)}")
print(f"\nModel Input Shape: {model.input_shape}")
print(f"Model Output Shape: {model.output_shape}")

## 2. Load the Trained Model and Weights

Load your pre-trained .h5 model from the `classifier/model/` directory.  
The model will automatically detect and load from:
- `CatDogBird_model.h5` (full model)
- `CatDogBird_model_Adv.h5` (advanced model)
- `ANN_model.weights.h5` (weights only)

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

# TensorFlow/Keras imports
import tensorflow as tf
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.efficientnet import preprocess_input

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

print("✓ Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")

## 1. Import Required Libraries

# Animal Classification Model - Prediction Notebook

This notebook demonstrates how to load your trained EfficientNetB0 model (.h5) and use it to make predictions on new animal images.

**Classes:** Bird, Cat, Dog  
**Model:** EfficientNetB0 + Dense layers  
**Image Size:** 224x224 pixels